In [3]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.


In [4]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}


In [5]:
import pandas as pd
import numpy as np

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    
    u_towns = pd.read_csv("university_towns.txt", sep="|", header=None)
    flag = 0
    u_towns_real = pd.DataFrame()
    for i in range (0,len(u_towns.index)):
        each_row = u_towns.iloc[i]
        if "[edit]" in each_row[0]:
            flag = 0
            state_name = each_row[0]
            #each_row[1] = state_name
        if (flag == 1):
            each_row[1] = each_row[0]
            each_row[0] = state_name
        if (flag == 0):
            flag = 1
        u_towns_real = u_towns_real.append(each_row)


    u_towns_real.dropna(inplace=True)

    st_names = list(u_towns_real[0].values)

    st_names_dict = {}

    for each_value in st_names:
        val = each_value.find('[')
        if (val != -1):
            res_val = each_value[0:val]
        else:
            res_val = each_value
        st_names_dict[each_value] = res_val

    u_towns_real[0].replace(st_names_dict, inplace=True)

    reg_names = list(u_towns_real[1].values)

    reg_names_dict = {}

    for each_value in reg_names:
        val = each_value.find('(')
        if (val != -1):
            res_val = each_value[0:val-1]
        else:
            res_val = each_value
        reg_names_dict[each_value] = res_val

    u_towns_real[1].replace(reg_names_dict, inplace=True)

    u_towns_real.columns = ['State','RegionName']

    u_towns_real.reset_index(inplace=True, drop=True)

    return u_towns_real


In [6]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    GDP = pd.read_excel("gdplev.xls")


    GDP.drop([x for x in range(0,219)], inplace=True)

    GDP.drop(['Unnamed: 3',GDP.columns[7]], axis=1, inplace=True)

    GDP.reset_index(inplace=True, drop=True)

    GDP.columns = ['Year','Year_GDP','Year_2009_GDP','Quarter','Quarter_GDP','Quarter_2009_GDP']

    for i in range(0,64):
        temp_df = GDP.iloc[i]
        first = temp_df['Quarter_2009_GDP']
        temp_df = GDP.iloc[i+1]
        second = temp_df['Quarter_2009_GDP']
        temp_df = GDP.iloc[i+2]
        third = temp_df['Quarter_2009_GDP']
        if (first > second > third):
            break

    return GDP.iloc[i+1]['Quarter']


In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    GDP = pd.read_excel("gdplev.xls")


    GDP.drop([x for x in range(0,219)], inplace=True)

    GDP.drop(['Unnamed: 3',GDP.columns[7]], axis=1, inplace=True)

    GDP.reset_index(inplace=True, drop=True)

    GDP.columns = ['Year','Year_GDP','Year_2009_GDP','Quarter','Quarter_GDP','Quarter_2009_GDP']

    recession_start = get_recession_start()

    pos = GDP[GDP['Quarter'] == recession_start].index.tolist()[0]

    for j in range(pos,66):
        temp_df = GDP.iloc[j]
        first1 = temp_df['Quarter_2009_GDP']
        temp_df = GDP.iloc[j+1]
        second1 = temp_df['Quarter_2009_GDP']
        temp_df = GDP.iloc[j+2]
        third1 = temp_df['Quarter_2009_GDP']
        if (first1 < second1 < third1):
            break

    return GDP.iloc[j+2]['Quarter']


In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    GDP = pd.read_excel("gdplev.xls")


    GDP.drop([x for x in range(0,219)], inplace=True)

    GDP.drop(['Unnamed: 3',GDP.columns[7]], axis=1, inplace=True)

    GDP.reset_index(inplace=True, drop=True)

    GDP.columns = ['Year','Year_GDP','Year_2009_GDP','Quarter','Quarter_GDP','Quarter_2009_GDP']

    recession_start = get_recession_start()
    recession_end = get_recession_end()

    start_pos = GDP[GDP['Quarter'] == recession_start].index.tolist()[0]
    end_pos = GDP[GDP['Quarter'] == recession_end].index.tolist()[0]

    min = 100000000
    for k in range(start_pos,end_pos+1):
        temp_df = GDP.iloc[k]
        if (temp_df['Quarter_2009_GDP']) < min:
            min = temp_df['Quarter_2009_GDP']
            min_index = k

    return GDP.iloc[min_index]['Quarter']


In [9]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    house = pd.read_csv("City_Zhvi_AllHomes.csv")

    col_to_delete = []

    for i in range(1997,2000):
        for j in range (1,13):
            str1 = str(i)
            str2 = str(j)
            if (len(str2) == 1):
                str2 = "0" + str2
            col_to_delete.append(str1+"-"+str2)

    pre_list = ['1996-04','1996-05','1996-06','1996-07','1996-08','1996-09','1996-10','1996-11','1996-12',]
    col_to_delete = pre_list + col_to_delete

    house.drop(col_to_delete, axis=1, inplace=True)
    house.replace(states, inplace=True)

    house.set_index(['State','RegionName'], inplace=True)

    house.drop(['RegionID','Metro','CountyName','SizeRank'], axis=1, inplace=True)
    
    
    count = 0
    len1 = len(house.columns.tolist())
    #print (len1)

    for i in range(2,len1,3):
        rows = [house.columns[i-2],house.columns[i-1],house.columns[i]]
        if ("-01" in house.columns[i-2]):
            append_str = "q1"
        elif ("-04" in house.columns[i-2]):
            append_str = "q2"
        elif ("-07" in house.columns[i-2]):
            append_str = "q3"
        else:
            append_str = "q4"
        prepend_str = house.columns[i-2][0:4]
        print (prepend_str, append_str)
        house[prepend_str+append_str] = house.apply(lambda x: np.mean(x[rows]), axis=1)

    rows1 = ['2016-07','2016-08']
    house['2016q3'] = house.apply(lambda x: np.mean(x[rows]), axis=1)
    
    house.drop(house.columns[0:200], axis=1, inplace=True)
  
    return house


In [10]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    recess_start = get_recession_start()
    recess_bottom = get_recession_bottom()
    
    house_new = convert_housing_data_to_quarters()

    cols = list(house_new.columns)
    start_ind = cols.index(recess_start)
    bottom_ind = cols.index(recess_bottom)

    house_upd = house_new[house_new.columns[start_ind:bottom_ind+1]]
    house_upd1 = house_upd.dropna()

    rows = [house_upd1.columns[0]]
    rows1 = [house_upd1.columns[3]]

    house_upd1['Ratio'] = house_upd1.apply(lambda x: np.divide(x[rows],x[rows1]), axis=1)
    
    uni_towns = get_list_of_university_towns()
    uni_towns1 = uni_towns.set_index(['State','RegionName'], drop=True)

    uni_towns_ratio = pd.merge(house_upd1, uni_towns1, how='inner', left_index=True, right_index=True)
    
    
    non_uni_towns_ratio = pd.concat([house_upd1,uni_towns_ratio]).drop_duplicates(keep=False)
    
    pval = ttest_ind(uni_towns_ratio['Ratio'],non_uni_towns_ratio['Ratio']).pvalue
    stats = ttest_ind(uni_towns_ratio['Ratio'],non_uni_towns_ratio['Ratio']).statistic
    
    if (pval < 0.01):
        different = True
    else:
        different = False

    if (stats < 0):
        better = "university town"
    else:
        better = "non-university town"

    tup = (different,pval,better)
    return tup
